### Quasi-Newton Methods
Quasi-Newton methods approximate the Hessian with a positive definite $B_k$ during each iteration, or they approximate the inverse of the Hessian with a matrix $H_k$.

An iteration of Quasi-Newton takes the form
$$\mathbf{x}_{k+1} = \mathbf{x}_k - \alpha_kB_k^{-1}\nabla f_k$$
Or
$$\mathbf{x}_{k+1} =  \mathbf{x}_k - \alpha_kH_k\nabla f_k$$

where $B_k$ is a positive definite approximation of the Hessian $\nabla^2 f_k$ and/or $H_k$ is a positive definite approximation of the inverse Hessian $[\nabla^2f_k]^{-1}$.

we choose $B_k$ such that it has the same property, that is determine a solution to

$$ B_{k+1}(\mathbf{x}_{k+1}-\mathbf{x}_k)= \nabla f_{k+1}-\nabla f_k,\qquad\text{or}\qquad B_{k+1}\mathbf{s}_k = \mathbf{y}_k $$

with $\mathbf{s}_k\equiv \mathbf{x}_{k+1}-\mathbf{x}_k$ and $\mathbf{y}_k\equiv\nabla f_{k+1}-\nabla f_k$. The above, called the **secant equation**, is equivalent to the following requirements for the inverse approximation, $H_{k+1}$, sometimes called the **dual secant equation**:

$$ H_{k+1}(\nabla f_{k+1}-\nabla f_k)=\mathbf{x}_{k+1}-\mathbf{x}_k,\qquad\text{or}\qquad H_{k+1}\mathbf{y}_k =\mathbf{s}_k $$


### Davidon-Fletcher-Powell (DFP) method

* Idea: Minimize $B_{k+1}$ such that
$$ B_{k+1} = \min_B \|B-B_k\|_{F,W}\qquad \text{subject to } B=B^T, B\mathbf{s}_k=\mathbf{y}_k $$

Results in
$$ B_{k+1} = \left(I-\frac{\mathbf{y}_k\mathbf{s}_k^T}{\mathbf{y}_k^T\mathbf{s}_k}\right)B_k\left(I-\frac{\mathbf{s}_k\mathbf{y}_k^T}{\mathbf{y}_k^T\mathbf{s}_k}\right) + \frac{\mathbf{y}_k\mathbf{y}_k^T}{\mathbf{y}_k^T\mathbf{s}_k} $$

But through the use of **Sherman-Morrison Formula**, we work with the inverse Hessian instead:

$$ H_{k+1} = B_{k+1}^{-1} = H_k + \frac{\mathbf{s}_k \mathbf{s}_k^T}{\mathbf{y}_k^{T} \mathbf{s}_k} - \frac{H_k \mathbf{y}_k \mathbf{y}_k^T H_k}{\mathbf{y}_k^T H_k \mathbf{y}_k} $$

Here, $y_k = \nabla f_{k+1} - \nabla f_k$, $s_k = x_{k+1} - x_k$.

**DFP can "get stuck"**: The inverse Hessian can become nearly singular (eigenvalue close to zero) - this will mean each update will not progress very far, even with optimal step size.

### Broyden-Fletcher-Goldfarb–Shanno (BFGS) method

* Idea: Minimize $H_{k+1}$ such that
$$ H_{k+1} = \min_H \|H-H_k\|_{F,W}\qquad \text{subject to } H=H^T, H\mathbf{y}_k=\mathbf{s}_k $$


The algorithm is

$$H_{k+1} = H_k + (\mathbf{y}_k^T\mathbf{s}_k+\mathbf{y}_k^T H_k \mathbf{y}_k)\frac{\mathbf{s}_k \mathbf{s}_k^T}{(\mathbf{y}_k^T \mathbf{s}_k)^2} - \frac{H_k \mathbf{y}_k \mathbf{s}_k^T + \mathbf{s}_k \mathbf{y}_k^TH_k}{\mathbf{y}_k^T\mathbf{s}_k} $$

**Convergence**: If $f$ is $\mathcal{C}^2$ (1st and 2nd derivatives are continuous) and BFGS sequence $x_k$ converges to a minimizer $x$
where the Hessian $D^2 f$ is Lipschitz continuous, and if 
$$\sum_{k\ge 1} \|x_k - x^*\|<\infty$$
then $x_k \rightarrow x^*$ at a **superlinear rate**.

* For both DFP and BFGS, main benefits are: $H_k$ preserves symmetry and positive definiteness, unlike SR-1.
* BFGS has more concrete convergence results than other Quasi-Newton methods, and inverse Hessian tends to avoid the issue of becoming nearly singular.
* On quadratic functions, all Quasi-Newton methods will take $n$ steps to converge (where $n$ is the dimension of the vector).

### Conjugate Gradient Method on Quadratic Functions

This version only works on quadratic functions of the form $f(x) = 1/2 x^T Q x - b^T x$.

1. Set $\mathbf{d}_0 = -\nabla f_0$, as in steepest descent.
2. Determine $\alpha_k$ and update $\mathbf{x}_{k+1} = \mathbf{x}_k + \alpha_k\mathbf{d}_k$.
3. Set $\mathbf{d}_{k+1} = -\nabla f_{k+1} + \beta_k\mathbf{d}_k$, where $\beta_k$ is chosen so that $\mathbf{d}_{k+1}$ and $\mathbf{d}_k$ are conjugate in some sense.
4. Repeat 2-3 until convergence.

If $f$ is quadratic, we showed the optimal values for the parameters $\alpha_k$ and $\beta_k$ were given by

$$ \alpha_k = -\frac{\mathbf{d}_k^T\nabla f_k}{\mathbf{d}_k^TQ\mathbf{d}_k},\qquad \beta_k = \frac{\mathbf{d}_k^TQ\nabla f_{k+1}}{\mathbf{d}_k^TQ\mathbf{d}_k} $$

In [55]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar

In [56]:
def WolfeI(alpha,f,x,dx,p,c1=0.1):
    '''Return True/False if Wolfe condition I is satisfied for the given alpha'''
    LHS = f(x[0]+alpha*p[0], x[1]+alpha*p[1])
    RHS = f(x[0],x[1])+c1*alpha*np.dot(dx,p)
    return LHS <= RHS

In [57]:
def CG_alg(x0, Q, b, tol=1e-7, max_steps=10000, rho=0.75):
    f = lambda x: 1/2 * x @ Q @ x - b @ x
    Df = lambda x: Q @ x - b
    x=x0
    path_CG = [x]
    tol = 1e-7            # stop when gradient is smaller than this amount
    max_steps = 10000     # Maximum number of steps to run the iteration
    rho = 0.75            # rho for backtracking
    i=0                   # iteration count
    gk = Df(x)          # current gradient
    dk = -gk
    dks = [dk]
    gks = [gk]

    while np.linalg.norm(gk)>tol and i<max_steps:    
        alpha = - dk @ gk / (dk @ Q @ dk)
        xnew = x + alpha*dk
        gk1 = Df(xnew)
        beta = (dk @ Q @ gk1)/ (dk @ Q @ dk)
        dk = -gk1+ beta*dk
        dks.append(dk)
        path_CG.append(xnew)
        x = xnew
        gk = gk1
        gks.append(gk)
        
        i += 1

    path_CG=np.array(path_CG)
    dks = np.array(dks)
    gks = np.array(gks)
    print(f'After {i} iterations of CG, approximate minimum is {f(x)} at {x}')
    return path_CG, dks, gks

In [58]:
def DFP(x0, Q, b, tol=1e-7, max_steps=10000, rho=0.75, exact=True):
    def f(x, y):
        x1 = np.array([x, y])
        return 1/2 * x1 @ Q @ x1 - b @ x1
    def Df(x, y):
        x1 = np.array([x, y])
        return Q @ x1 - b
    x,y = x0[0], x0[1]    # initial point
    path_DFP = [[x,y]]
    i=0                   # iteration count
    rho = 0.75            # rho for backtracking
    H = np.eye(2)         # initial inverse Hessian is identity
    dx = Df(x,y)          # current gradient
    while np.linalg.norm(dx)>tol and i<max_steps:
        pk = -H@dx
        if exact:
            def subproblem1D(alpha):
                return f(x + alpha * pk[0], y + alpha*pk[1])
            res = minimize_scalar(subproblem1D) # scipy function to minimize objFunction w.r.t alpha
            print("Exact line search step size: {:.4f}".format(res.x))
            alpha = res.x
        else:
            # backtracking
            alpha = 1
            while not WolfeI(alpha,f,np.array([x,y]),dx,pk) and alpha>1e-5:  # lower limit to prevent small steps, similar to Wolfe II
                alpha *= rho
        xnew,ynew = x + alpha*pk[0], y + alpha*pk[1]
        path_DFP.append([xnew,ynew])

        # secant variables
        sk = alpha*pk         # x_{k+1}-x_k
        yk = Df(xnew,ynew)-dx # Df_{k+1}-Df_k

        # DFP update
        vec = H@yk # @ is matrix multiplication
        denom1 = yk@sk
        denom2 = yk@vec
        H += np.outer(sk,sk)/denom1 - np.outer(vec,vec)/denom2 # np.outer(vec, vec) = vec * vec^T

        x,y = xnew,ynew
        dx = Df(x,y)
        i += 1
    path_DFP=np.array(path_DFP)
    print(f'After {i} iterations of DFP, approximate minimum is {f(x,y)} at {x,y}')

In [59]:
%%time

def BFGS(x0, Q, b, tol=1e-7, max_steps=10000, rho=0.75, exact=True):
    def f(x, y):
        x1 = np.array([x, y])
        return 1/2 * x1 @ Q @ x1 - b @ x1
    def Df(x, y):
        x1 = np.array([x, y])
        return Q @ x1 - b
    x,y = x0[0], x0[1]    # initial point
    path_BFGS = [[x,y]]
    i=0                   # iteration count
    rho = 0.75            # rho for backtracking
    H = np.eye(2)         # initial inverse Hessian is identity
    dx = Df(x,y)          # current gradient
    while np.linalg.norm(dx)>tol and i<max_steps:
        pk = -H@dx
        if exact:
            def subproblem1D(alpha):
                return f(x + alpha * pk[0], y + alpha*pk[1])
            res = minimize_scalar(subproblem1D) # scipy function to minimize objFunction w.r.t alpha
            print("Exact line search step size: {:.4f}".format(res.x))
            alpha = res.x
        else:
            # backtracking
            alpha = 1
            while not WolfeI(alpha,f,np.array([x,y]),dx,pk) and alpha>1e-5:  # lower limit to prevent small steps, similar to Wolfe II
                alpha *= rho
        xnew,ynew = x + alpha*pk[0], y + alpha*pk[1]
        path_BFGS.append([xnew,ynew])

        # secant variables
        sk = alpha*pk         # x_{k+1}-x_k
        yk = Df(xnew,ynew)-dx # Df_{k+1}-Df_k

        # BFGS update
        vec = H@yk
        denom = yk@sk
        H += (denom+vec@yk)*np.outer(sk,sk)/denom**2 - (np.outer(vec,sk)+np.outer(sk,vec))/denom

        x,y = xnew,ynew
        dx = Df(x,y)
        i += 1

    path_BFGS=np.array(path_BFGS)
    print(f'After {i} iterations of BFGS, approximate minimum is {f(x,y)} at {x,y}')

Wall time: 1e+03 µs


Just to demonstrate convergence properties on Quadratic Functions:

In [60]:
Q = np.array([[9,0],[0,4]])
b = np.array([2,4])
x0 = (1.5,2)
DFP(x0,Q,b,exact=True)
BFGS(x0,Q,b,exact=True)
CG_alg(x0,Q,b)

Exact line search step size: 0.1182
Exact line search step size: 0.2429
After 2 iterations of DFP, approximate minimum is -2.2222222222222228 at (0.22222222222222213, 1.0000000000000002)
Exact line search step size: 0.1182
Exact line search step size: 0.2350
After 2 iterations of BFGS, approximate minimum is -2.222222222222222 at (0.22222222343190004, 0.9999999921748961)
After 2 iterations of CG, approximate minimum is -2.2222222222222228 at [0.22222222 1.        ]


(array([[1.5       , 2.        ],
        [0.14072155, 1.52720749],
        [0.22222222, 1.        ]]), array([[-1.15000000e+01, -4.00000000e+00],
        [ 3.46796177e-01, -2.24333777e+00],
        [ 6.32143331e-16,  2.19875941e-16]]), array([[ 1.15000000e+01,  4.00000000e+00],
        [-7.33506079e-01,  2.10882998e+00],
        [-6.66133815e-16,  0.00000000e+00]]))

### Efficient CG Algorithm

In [61]:
def CG_alg_efficient(x0, Q, b, tol=1e-7, max_steps=10000, rho=0.75):
    f = lambda x: 1/2 * x @ Q @ x - b @ x
    Df = lambda x: Q @ x - b
    x=x0
    path_CG = [x]
    tol = 1e-7            # stop when gradient is smaller than this amount
    max_steps = 10000     # Maximum number of steps to run the iteration
    rho = 0.75            # rho for backtracking
    i=0                   # iteration count
    
    rk = Q @ x0 - b
    dk = -rk
    dks = [dk]

    while np.linalg.norm(rk)>tol and i<max_steps:    
        yk = Q @ dk
        alpha = rk @ rk / (dk @ yk)
        rk1 = rk + alpha * yk # Step 4
        xnew = x + alpha*dk
        beta = (rk1 @ rk1)/ (rk @ rk)
        dk = -rk1+ beta*dk
        dks.append(dk)
        path_CG.append(xnew)
        x = xnew        
        rk = rk1
        i += 1

    path_CG=np.array(path_CG)
    dks = np.array(dks)
    print(f'After {i} iterations, approximate minimum of efficient CG is {f(x)} at {x}')
    return path_CG, dk

Comparing the time for efficient CG vs regular CG implementation:

In [62]:
import time
matrixSize = 10
Q = np.random.rand(matrixSize, matrixSize)
Q = np.dot(Q, Q.transpose())
b = np.random.rand(matrixSize)
x0 = np.random.rand(matrixSize)
start_time = time.time()
CG_alg_efficient(x0,Q,b)
end_time = time.time() - start_time
print("Efficient CG takes {:.5f}s".format(end_time))
start_time = time.time()
CG_alg(x0,Q,b)
end_time = time.time() - start_time
print("CG takes {:.5f}s".format(end_time))

After 11 iterations, approximate minimum of efficient CG is -2.130254861020093 at [ -4.96960792  -4.3496964   -7.20044619   9.39735763   2.40299444
   5.11442835 -13.27780009   2.22084286   3.99167697   5.69108672]
Efficient CG takes 0.00100s
After 11 iterations of CG, approximate minimum is -2.1302548610200236 at [ -4.96960792  -4.3496964   -7.20044619   9.39735763   2.40299444
   5.11442835 -13.27780009   2.22084286   3.99167697   5.69108672]
CG takes 0.00000s
